Importing applicationInformation

In [1]:
import pandas as pd
import datetime
pd.set_option('display.max_column', None)
game_data = pd.read_csv('applicationInformation.csv', 
                        encoding = 'latin-1',
                    )
#Converting freetoplay to boolean
game_data.freetoplay = game_data.freetoplay.replace({1.0: True, 0.0: False}) 


In [2]:
#Cleaning Colnames
game_data.rename(columns = {'appid': 'app_id',
                           'releasedate' : 'release_date',
                           'freetoplay' : 'free_to_play'},
                            inplace = True)



#release_date to datetime
game_data['release_date'] = pd.to_datetime(game_data['release_date'],format = '%d-%b-%y', errors = 'coerce')



Combining with Application Developers

In [3]:
game_devs = pd.read_csv('applicationDevelopers.csv',
                        names = ['app_id', 'developer_1', 'developer_2', 'developer_3','developer_4','developer_5','developer_6','developer_7'],
                        encoding = 'latin-1')
#Drop to just 3 devs
game_devs = game_devs.iloc[:,0:4]
game_data = pd.merge(game_data, game_devs, on = 'app_id', how = 'inner')
game_data

,app_id,type,name,release_date,free_to_play,developer_1,developer_2,developer_3
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,False,PUBG Corporation,NaN,NaN
1,570,game,Dota 2,2013-07-09,True,Valve,NaN,NaN
2,730,game,Counter-Strike: Global Offensive,2012-08-21,True,Valve,Hidden Path Entertainment,NaN
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,False,Ubisoft Montreal,NaN,NaN
...,...,...,...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,2009-10-29,True,Sports Interactive,NaN,NaN
1996,202480,NaN,Skyrim Creation Kit,NaT,NaN,NaN,NaN,NaN
1997,429050,game,Feed and Grow: Fish,2016-01-08,False,Old B1ood,NaN,NaN
1998,209650,game,Call of Duty: Advanced Warfare,2014-11-03,False,Sledgehammer Games,Raven Software,NaN


Combining w/ Publishers

In [4]:
game_pub = pd.read_csv('applicationPublishers.csv',
                      names = ['app_id', 'publisher_1', 'publisher_2', 'publisher_3', 'publisher_4'],
                      encoding = 'latin-1'
                      )

#Drop to just 3 pubs
game_pub = game_pub.iloc[:,0:4]
game_data = pd.merge(game_data, game_pub, on = 'app_id', how = 'inner')
game_data

,app_id,type,name,release_date,free_to_play,developer_1,developer_2,developer_3,publisher_1,publisher_2,publisher_3
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,False,PUBG Corporation,NaN,NaN,PUBG Corporation,NaN,NaN
1,570,game,Dota 2,2013-07-09,True,Valve,NaN,NaN,Valve,NaN,NaN
2,730,game,Counter-Strike: Global Offensive,2012-08-21,True,Valve,Hidden Path Entertainment,NaN,Valve,NaN,NaN
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,False,Ubisoft Montreal,NaN,NaN,Ubisoft,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,2009-10-29,True,Sports Interactive,NaN,NaN,SEGA,NaN,NaN
1996,202480,NaN,Skyrim Creation Kit,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,429050,game,Feed and Grow: Fish,2016-01-08,False,Old B1ood,NaN,NaN,Greens s.r.o.,NaN,NaN
1998,209650,game,Call of Duty: Advanced Warfare,2014-11-03,False,Sledgehammer Games,Raven Software,NaN,Activision,NaN,NaN


One Hot Encoding Genres

In [5]:
#Reading and setting up short df
game_genres_long = pd.read_csv('applicationGenres.csv',
                      names = ['app_id', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7', 'genre_8'],
                      encoding = 'latin-1',
                      index_col = False
                      )
game_genres = pd.DataFrame(columns = ['app_id', 'genre'])
for index, row in game_genres_long.iterrows():
    for c in range(1,7):
        if(pd.isnull(row[c])):
            c = 8
        else:
            app_id = row[0] #Getting appid
            genre = row[c] #Getting genre
            game_genres.loc[len(game_genres.index)] = [app_id, genre] #Adding to short df 
            


In [6]:
#One-Hot Encoding
game_genres_final = pd.get_dummies(game_genres, columns = ['genre'], prefix='genre')
game_genres_final.drop('genre_60', axis = 1, inplace = True)
game_genres_final = game_genres_final.groupby('app_id', sort = False).sum()
game_genres_final

,genre_Action,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Early Access,genre_Education,genre_Free to Play,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing
app_id,,,,,,,,,,,,,,,,,,,,,,,,,
578080,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
570,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
730,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
359550,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
271590,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
227860,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
429050,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [7]:
#Getting genre names
col_names = list(game_genres_final)

In [8]:
#Merging into master df
game_data = pd.merge(game_data, game_genres_final, on = 'app_id', how = 'left')
game_data[col_names] = game_data[col_names].replace({1.0: True, 0.0: False}) #Convert to True/False
game_data

,app_id,type,name,release_date,free_to_play,developer_1,developer_2,developer_3,publisher_1,publisher_2,publisher_3,genre_Action,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Early Access,genre_Education,genre_Free to Play,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,False,PUBG Corporation,NaN,NaN,PUBG Corporation,NaN,NaN,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,570,game,Dota 2,2013-07-09,True,Valve,NaN,NaN,Valve,NaN,NaN,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2,730,game,Counter-Strike: Global Offensive,2012-08-21,True,Valve,Hidden Path Entertainment,NaN,Valve,NaN,NaN,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,False,Ubisoft Montreal,NaN,NaN,Ubisoft,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,2009-10-29,True,Sports Interactive,NaN,NaN,SEGA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,202480,NaN,Skyrim Creation Kit,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,429050,game,Feed and Grow: Fish,2016-01-08,False,Old B1ood,NaN,NaN,Greens s.r.o.,NaN,NaN,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1998,209650,game,Call of Duty: Advanced Warfare,2014-11-03,False,Sledgehammer Games,Raven Software,NaN,Activision,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Added PlayerCounts

In [9]:
mean_players = pd.read_csv('mean_player_counts.csv', index_col=0)
mean_players = mean_players.rename(columns = {'appid': 'app_id'})
game_data = pd.merge(game_data, mean_players, on = 'app_id', how = 'left')
game_data

,app_id,type,name,release_date,free_to_play,developer_1,developer_2,developer_3,publisher_1,publisher_2,publisher_3,genre_Action,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Early Access,genre_Education,genre_Free to Play,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing,meanPlayers2017,meanPlayers2018,meanPlayers2019,meanPlayers2020
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,False,PUBG Corporation,NaN,NaN,PUBG Corporation,NaN,NaN,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,1443309,841204,366385,238828
1,570,game,Dota 2,2013-07-09,True,Valve,NaN,NaN,Valve,NaN,NaN,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,528727,450816,475222,442868
2,730,game,Counter-Strike: Global Offensive,2012-08-21,True,Valve,Hidden Path Entertainment,NaN,Valve,NaN,NaN,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,347535,317709,396300,660862
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2811,580,587
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,False,Ubisoft Montreal,NaN,NaN,Ubisoft,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,60281,67572,71262,90971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,2009-10-29,True,Sports Interactive,NaN,NaN,SEGA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102,88,70,89
1996,202480,NaN,Skyrim Creation Kit,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153,138,101,91
1997,429050,game,Feed and Grow: Fish,2016-01-08,False,Old B1ood,NaN,NaN,Greens s.r.o.,NaN,NaN,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,225,280,258,304
1998,209650,game,Call of Duty: Advanced Warfare,2014-11-03,False,Sledgehammer Games,Raven Software,NaN,Activision,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,111,87,72,66


Price Data

In [10]:
import numpy as np

# priceHistoryDict{appid : (appid, firstAvailiblePrice, latestAvailiblePrice, lowestPrice, highestPrice)}
priceHistoryDict = {}

def getPriceSummary(appid):
    global priceHistoryDict
    #path = f"../data/PriceHistory/PriceHistory/{appid}.csv" #Evan's file structure
    path = f"PriceHistory/{appid}.csv" #Jon's file structure
    try:
        priceHistoryData = pd.read_csv(f"PriceHistory/{appid}.csv",
                                  index_col = False,
                                  encoding = 'latin-1')
    
        
    except FileNotFoundError:
        print(f"ERROR! {appid}.csv not found!")
        priceHistoryDict[appid] = (appid, 0, 0, 0, 0)
        return
    priceHistoryDict[appid] = (appid,
                                priceHistoryData["Initialprice"].iloc[0], #firstAvailiblePrice
                                priceHistoryData["Finalprice"].iloc[-1], #latestAvailiblePrice
                                priceHistoryData["Finalprice"].min(),
                                priceHistoryData["Finalprice"].max()
                              )
    
    
game_data["app_id"].map(getPriceSummary)


ERROR! 570.csv not found!
ERROR! 730.csv not found!
ERROR! 622590.csv not found!
ERROR! 238960.csv not found!
ERROR! 230410.csv not found!
ERROR! 440.csv not found!
ERROR! 433850.csv not found!
ERROR! 444090.csv not found!
ERROR! 482730.csv not found!
ERROR! 304930.csv not found!
ERROR! 504370.csv not found!
ERROR! 236390.csv not found!
ERROR! 363970.csv not found!
ERROR! 601510.csv not found!
ERROR! 386360.csv not found!
ERROR! 444200.csv not found!
ERROR! 480.csv not found!
ERROR! 550650.csv not found!
ERROR! 291550.csv not found!
ERROR! 378120.csv not found!
ERROR! 227940.csv not found!
ERROR! 698780.csv not found!
ERROR! 259360.csv not found!
ERROR! 291480.csv not found!
ERROR! 386180.csv not found!
ERROR! 304050.csv not found!
ERROR! 627690.csv not found!
ERROR! 453480.csv not found!
ERROR! 8500.csv not found!
ERROR! 301520.csv not found!
ERROR! 552990.csv not found!
ERROR! 466240.csv not found!
ERROR! 295270.csv not found!
ERROR! 109600.csv not found!
ERROR! 738060.csv not found!

ERROR! 439700.csv not found!
ERROR! 457330.csv not found!
ERROR! 391460.csv not found!
ERROR! 630.csv not found!
ERROR! 588220.csv not found!
ERROR! 7670.csv not found!
ERROR! 723790.csv not found!
ERROR! 217750.csv not found!
ERROR! 215830.csv not found!
ERROR! 317470.csv not found!
ERROR! 649900.csv not found!
ERROR! 476460.csv not found!
ERROR! 581130.csv not found!
ERROR! 354290.csv not found!
ERROR! 339280.csv not found!
ERROR! 757570.csv not found!
ERROR! 697610.csv not found!
ERROR! 438140.csv not found!
ERROR! 317400.csv not found!
ERROR! 510630.csv not found!
ERROR! 568770.csv not found!
ERROR! 34270.csv not found!
ERROR! 451130.csv not found!
ERROR! 619310.csv not found!
ERROR! 354830.csv not found!
ERROR! 619910.csv not found!
ERROR! 624910.csv not found!
ERROR! 206210.csv not found!
ERROR! 17520.csv not found!
ERROR! 321400.csv not found!
ERROR! 380840.csv not found!
ERROR! 510660.csv not found!
ERROR! 466910.csv not found!
ERROR! 381990.csv not found!
ERROR! 522230.csv not

0       None
1       None
2       None
3       None
4       None
        ... 
1995    None
1996    None
1997    None
1998    None
1999    None
Name: app_id, Length: 2000, dtype: object

In [11]:
game_data = pd.merge(game_data, 
                     pd.DataFrame.from_dict(priceHistoryDict, orient="index", 
                            columns = ["app_id", 
                                       "first_available_price", 
                                       "last_available_price", 
                                       "lowest_price", 
                                       "highest_price"]), 
                     on = 'app_id', 
                     how = 'left')
game_data.head()



,app_id,type,name,release_date,free_to_play,developer_1,developer_2,developer_3,publisher_1,publisher_2,publisher_3,genre_Action,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Early Access,genre_Education,genre_Free to Play,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing,meanPlayers2017,meanPlayers2018,meanPlayers2019,meanPlayers2020,first_available_price,last_available_price,lowest_price,highest_price
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,False,PUBG Corporation,NaN,NaN,PUBG Corporation,NaN,NaN,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,1443309,841204,366385,238828,29.99,29.99,14.99,29.99
1,570,game,Dota 2,2013-07-09,True,Valve,NaN,NaN,Valve,NaN,NaN,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,528727,450816,475222,442868,0.00,0.00,0.00,0.00
2,730,game,Counter-Strike: Global Offensive,2012-08-21,True,Valve,Hidden Path Entertainment,NaN,Valve,NaN,NaN,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,347535,317709,396300,660862,0.00,0.00,0.00,0.00
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2811,580,587,0.00,0.00,0.00,0.00
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,False,Ubisoft Montreal,NaN,NaN,Ubisoft,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,60281,67572,71262,90971,19.99,19.99,7.99,19.99


In [15]:
#Changing developer/publisher NAs to 0s
replace_values = {"developer_2": 0, "developer_3": 0, "publisher_2": 0, "publisher_3":0}
data = game_data.fillna(value = replace_values)
data = data.replace({True:1, False:0}) #Change true/false to 1/0
#data = data.dropna() #Remove NA rows
data


,app_id,type,name,release_date,free_to_play,developer_1,developer_2,developer_3,publisher_1,publisher_2,publisher_3,genre_Action,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Early Access,genre_Education,genre_Free to Play,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing,meanPlayers2017,meanPlayers2018,meanPlayers2019,meanPlayers2020,first_available_price,last_available_price,lowest_price,highest_price
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,0.0,PUBG Corporation,0,0,PUBG Corporation,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1443309,841204,366385,238828,29.99,29.99,14.99,29.99
1,570,game,Dota 2,2013-07-09,1.0,Valve,0,0,Valve,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,528727,450816,475222,442868,0.00,0.00,0.00,0.00
2,730,game,Counter-Strike: Global Offensive,2012-08-21,1.0,Valve,Hidden Path Entertainment,0,Valve,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,347535,317709,396300,660862,0.00,0.00,0.00,0.00
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2811,580,587,0.00,0.00,0.00,0.00
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,0.0,Ubisoft Montreal,0,0,Ubisoft,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60281,67572,71262,90971,19.99,19.99,7.99,19.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,2009-10-29,1.0,Sports Interactive,0,0,SEGA,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102,88,70,89,0.00,0.00,0.00,0.00
1996,202480,NaN,Skyrim Creation Kit,NaT,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153,138,101,91,0.00,0.00,0.00,0.00
1997,429050,game,Feed and Grow: Fish,2016-01-08,0.0,Old B1ood,0,0,Greens s.r.o.,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225,280,258,304,14.99,14.99,8.99,14.99
1998,209650,game,Call of Duty: Advanced Warfare,2014-11-03,0.0,Sledgehammer Games,Raven Software,0,Activision,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111,87,72,66,59.99,59.99,14.99,59.99


In [16]:
data.to_csv('cleaned_game_data.csv')